#### Price Action Indicators

In [1]:
import os
import sqlite3
import pandas as pd
import numpy as np
from finta import TA

import matplotlib as plot
import mplfinance as mpf
import plotly.graph_objects as go

#%matplotlib inline

database_name = "TSX_Quality.sqlite"

#### Load our Data  

In [2]:
# Load from price table
# Convert string columns to float values
# Replace "-" prices with Close values
# Make sure al dates are sorted in ascending order (TA libraries need this order)

In [3]:
# Heavy function to retreive all prices from database (more than 6 million rows, takes many seconds to execute)
def get_all_prices():
    conn = sqlite3.connect(database_name)
    
    # Make sur all dates are in ascending order
    sql = f"SELECT * FROM prices_daily ORDER BY UPPER(Ticker) ASC, Date ASC"
    prices = pd.read_sql_query(sql, conn, index_col="Date")
    prices.index = pd.to_datetime(prices.index)
    
    # Clean up CSV data to make sure we have only floats and no "-" values
    prices.replace("-", np.NaN, inplace=True)
    #prices["Volume"].replace(0, np.NaN, inplace=True)
    prices["Open"] = prices["Open"].astype(float)
    prices["High"] = prices["High"].astype(float)
    prices["Low"]  = prices["Low"].astype(float)
    prices["Close"]= prices["Close"].astype(float)
    
    # Required for finTA
    prices.rename(columns={"Open":"open", "High":"high", "Low":"low", "Close":"close", "Volume":"volume"}, inplace=True)
    prices.index.rename("date", inplace=True)

    prices_per_ticker = prices.groupby(["Ticker"])

    return prices_per_ticker

def get_all_symbols():
    conn = sqlite3.connect(database_name)
    sql = f"SELECT * FROM symbols ORDER BY UPPER(Ticker) ASC"
    symbols_df = pd.read_sql_query(sql, conn)
    
    return symbols_df

prices_df = get_all_prices()
# Access specific company using .get_group("TickerSymbol")


In [4]:
# prices_per_ticker.get_group("SHOP").loc["2021-03" : "2021-05"]   #Deprecated method sort dataframe before using slicing on index
# prices_per_ticker.get_group("SHOP").sort_index().loc["2016-03" : "2016-05"]
prices_df.get_group("SHOP")

,Ticker,open,high,low,close,volume
date,,,,,,
2014-08-26,SHOP,NaN,NaN,NaN,0.040000,0.0
2014-08-27,SHOP,NaN,NaN,NaN,0.040000,0.0
2014-08-28,SHOP,NaN,NaN,NaN,0.040000,0.0
2014-08-29,SHOP,NaN,NaN,NaN,0.040000,0.0
2014-09-01,SHOP,NaN,NaN,NaN,0.040000,0.0
...,...,...,...,...,...,...
2022-01-11,SHOP,1388.209961,1466.369995,1375.000000,1456.770020,251900.0
2022-01-12,SHOP,1487.459961,1489.000000,1449.030029,1461.849976,252800.0
2022-01-13,SHOP,1468.989990,1468.989990,1329.930054,1333.060059,281900.0


In [5]:
# prices_per_ticker.get_group("SHOP").sort_index().loc["2015"]["High"].max()
prices_df.get_group("SHOP").sort_index().loc["2020"]["low"].min()
# 2228.73  : MAX All years      MIN   25.85
# 1742.37  : MAX 2022           MIN 1424.95
# 2228.73  : MAX 2021           MIN 1248.55
# 1658.79  : MAX 2020           MIN  435.03
#  544.00  : MAX 2019           MIN  174.59
#  232.65  : MAX 2018           MIN  126.65
#  151.88  : MAX 2017           MIN   56.60
#   60.93  : MAX 2016           MIN   25.85
#   53.50  : MAX 2015           MIN   30.00

435.03

In [6]:
# prices_2 = prices_df.get_group("SHOP").copy()
# prices_2["SMA"] = prices_2["close"].rolling(window=5).mean()
# prices_2.tail(5)

In [7]:
def run_indicators(prices):
    #prices_per_ticker.get_group(symbol)
    data_df = prices.copy()
    data_df.drop("Ticker", axis=1, inplace=True)
    #data_df.rename(columns={"Open":"open", "High":"high", "Low":"low", "Close":"close", "Volume":"volume"}, inplace=True)
    #data_df.index.rename("date", inplace=True)
    #data_df.sort_index(ascending=True, inplace=True)
    
    indicators_df = pd.DataFrame()
    # Return a series
    try:
        for period in ["200", "100", "50", "30", "20", "15", "9", "5"]:
            indicators_df[f"SMA{period}"] = TA.SMA(data_df, int(period))
    except Exception:
        print(f"Error calculating SMA{period}")
    try:
        for period in ["200", "100", "50", "30", "20", "15", "9", "5"]:
            indicators_df[f"EMA{period}"] = TA.EMA(data_df, int(period))
    except Exception:
        print(f"Error calculating EMA{period}")
    try:
        for period in ["200", "100", "50", "30", "20", "15", "9", "5"]:
            indicators_df[f"VAMA{period}"] = TA.VAMA(data_df, int(period))
    except Exception:
        print(f"Error calculating VAMA{period}")
    try:
        for period in ["200", "100", "50", "30", "20", "15", "9", "5"]:
            indicators_df[f"WMA{period}"] = TA.WMA(data_df, int(period))
    except Exception:
        print(f"Error calculating WMA{period}")
    try:
        for period in ["30", "20", "14", "10", "7"]:
            indicators_df[f"RSI{period}"] = TA.RSI(data_df, int(period))
    except Exception:
        print(f"Error calculating RSI{period}")
    try:
        for period in ["30", "20", "14", "10", "7"]:
            indicators_df[f"ATR{period}"] = TA.ATR(data_df, int(period))
    except Exception:
        print(f"Error calculating ATR{period}")        

    data_df = data_df.join(indicators_df)

    # Returns a Dataframe
    try:
        MACD = TA.MACD(data_df)
        MACD.rename(columns={"SIGNAL":"MACD_SIGNAL"}, inplace=True)
        data_df = data_df.join(MACD)
    except Exception:
        print(f"Error calculating MACD")

    try:
        VW_MACD = TA.MACD(data_df)
        VW_MACD.rename(columns={"MACD":"VW_MACD"}, inplace=True)
        VW_MACD.rename(columns={"SIGNAL":"VW_MACD_SIGNAL"}, inplace=True)    
        data_df = data_df.join(VW_MACD)
    except Exception:
        print(f"Error calculating VW_MACD (Value Weighted MACD)")

    try:
        BBANDS = TA.BBANDS(data_df) # Standard 20 day period, with 2 standard deviations
        data_df = data_df.join(BBANDS)
    except Exception:
        print(f"Error calculating BBANDS (Bolinger Bands)")

    return data_df

#run_indicators(prices_df.get_group("SHOP") )


In [10]:
def run_indicators_for_all_symbols(prices_df):
    symbols_df = get_all_symbols()
    symbols_df = symbols_df.dropna()
    conn = sqlite3.connect(database_name)
    for symbol in symbols_df["ticker"]:
        print(symbol)
        try:
            prices = prices_df.get_group(symbol)        
            data = run_indicators(prices)  
        except Exception:
            print(f"{symbol} : Not found in DB")

        try:
            #data["ticker"] = symbol
            data.insert(0,"ticker", symbol, allow_duplicates=True)
            data.drop(["open","high","low","close","volume"], axis=1, inplace=True)            
        except Exception:
            print(f"{symbol}: Drop columns went wrong")    

        try:
            #data.to_csv("indicators.csv", mode="a", index=False)
            data.to_sql("Indicators", conn, if_exists='append', index=True)
        except Exception:
            print(f"{symbol} : Problems saving data to file or DB")

    conn.close()

run_indicators_for_all_symbols(prices_df)

A
AAA.P
AAB


In [9]:
def strategy_1(prices):
    """ Rayner Teo : https://www.youtube.com/watch?v=W8ENIXvcGlQ
    CONDITIONS:
    1) Close price > SMA200
    2) RSI10 < 30
    3) Next Day = BUY
    """
    data = run_indicators(prices)

    data["UpTrend"] = data["close"] > data["SMA200"]
    data["DownTrend"] = data["close"] <= data["SMA200"]

    return data

